In [1]:
import pandas as pd
from pandas import read_csv


In [2]:
def read_csv(folder):
    #needs args
    train_df = pd.read_csv(folder + 'split/train.csv')
    val_df = pd.read_csv(folder + 'split/validation.csv')
    test_df = pd.read_csv(folder + 'split/test.csv')
    return train_df, val_df, test_df 

In [3]:

train_df, val_df, test_df = read_csv('data/')

In [4]:
from nltk.corpus import stopwords

In [5]:
def fetchvocab(trainfile):
    df = trainfile
    
    stop_words = set(stopwords.words('english'))
    tokenize = lambda x: word_tokenize(x)

    print('Removing stopwords')
    for colname in ['sent1', 'sent2']:
        df[colname] = df[colname].str.lower().str.split() #t his could be tokenized in a better way of course :)
        #df[colname] = df[colname].apply(lambda x: [item for item in x if item not in stop_words])
    print('Getting sentences and vocab')
    vocab = set()
    sents = []
    for idx, row in df.iterrows():
        sent1 = row['sent1']
        vocab.update(sent1)
        sent2 = row['sent2']
        vocab.update(sent2)
        label = row['label']
        sents.append([sent1, sent2, label])
    return vocab

def encode_train_data(df, char2int):
    stop_words = set(stopwords.words('english'))
    for colname in ['sent1', 'sent2']:
        df[colname] = df[colname].apply(lambda x: [char2int[i] for i in x])
                

In [6]:
vocab = fetchvocab(train_df)

Removing stopwords
Getting sentences and vocab


In [7]:
int2char = dict(enumerate(vocab))
char2int = {char : num for num, char in int2char.items()}

In [21]:
encoded = encode_train_data(train_df, char2int)

In [9]:
def zipping_data(df):
    X_train = []
    y_train = []
    for idx, row in df.iterrows():
        sent_1 = row['sent1']
        sent_2 = row['sent2']
        X_train.append((sent_1,sent_2))
        y_train.append(row['label'])
        
    return X_train, y_train
       
        

In [151]:
X_train, Y_train = zipping_data(train_df)

In [23]:
from torch.utils.data import Dataset, DataLoader

In [152]:
sent1 = [i[0] for i in X_train]

In [153]:
sent2 = [i[1] for i in X_train]

In [26]:
class DebatesSet(Dataset):
    def __init__(self,sent1,sent2, y):
        self.sent1 = sent1
        self.sent2 = sent2
        self.y = y
        
    def __getitem__(self, index):
        x = self.sent1[index]
        y = self.sent2[index]
        z = self.y[index]
        
        return x,y, z
        
    def __len__(self):
        return len(self.y)

In [154]:
class sent(Dataset):
    def __init__(self,x, y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        
        return x,y
        
    def __len__(self):
        return len(self.y)

In [155]:
train_data = DebatesSet(sent1,sent2,Y_train)

[[7642,
  80035,
  44519,
  97315,
  89879,
  79895,
  89695,
  56711,
  22098,
  110504,
  76834,
  48504,
  97315,
  102344,
  62891,
  89695,
  113067,
  18804],
 [7642,
  34788,
  112318,
  52637,
  22098,
  56690,
  44202,
  65835,
  46382,
  140753,
  97315,
  56961,
  46382,
  112318,
  78745],
 [117461,
  48504,
  61341,
  133863,
  97315,
  109157,
  31257,
  72599,
  141102,
  132311,
  8633,
  79698,
  12720,
  22098,
  97315,
  48196,
  119429,
  125299,
  132311,
  87537,
  56069,
  79900,
  90840,
  100176,
  82110,
  6271,
  46825,
  91820,
  22098,
  145799],
 [97315,
  89643,
  115796,
  132311,
  70800,
  91116,
  64563,
  97315,
  129673,
  48504,
  97315,
  46451,
  53527,
  97315,
  41170,
  110451,
  6271,
  121580,
  97471,
  115793,
  82457,
  13097,
  97315,
  117401,
  97444,
  146771],
 [113275,
  29648,
  111086,
  10766,
  97315,
  122794,
  23479,
  48504,
  40442,
  97315,
  18804],
 [88661,
  138861,
  137748,
  122323,
  119675,
  97315,
  15951,
  6800

In [157]:
sent1_dataset = sent(sent1, Y_train)

In [158]:
sent2_dataset = sent(sent2, Y_train)

In [209]:
sent1_dataset[1]

([7642,
  34788,
  112318,
  52637,
  22098,
  56690,
  44202,
  65835,
  46382,
  140753,
  97315,
  56961,
  46382,
  112318,
  78745],
 1)

In [226]:
sent1_load = DataLoader(sent1_dataset, batch_size=20, shuffle=False)
sent2_load = DataLoader(sent2_dataset, batch_size=20, shuffle=False)

In [233]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, batch_size, weights, drop_prob=0.2, device = 'cpu'):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.device = torch.device(device)
        self.batch_size = batch_size

        self.embedding = nn.Embedding.from_pretrained(weights)

        self.gru1 = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.gru2 = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)

    def forward(self, sent1, sent2):
        h1_init = self.init_hidden(self.batch_size)

        sent1 = self.embedding(sent1)
        out1, h1 = self.gru1(sent1, h1_init)
        h1 = F.relu(h1)

        h2_init = self.init_hidden(self.batch_size)

        sent2 = self.embedding(sent2)
        out2, h2 = self.gru2(sent2, h2_init)
        h2 = F.relu(h2)

        h = torch.cat(h1,h2)

        out = self.fc1(h)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.out(out)
        out = nn.Sigmoid(out)

        return out

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(self.device)
        return hidden

In [227]:
import numpy as np

In [232]:
for (sent1_dat, sent1_lab), (sent2_dat,sent2_lab) in zip(sent1_load,sent2_load):
    

tensor([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]) tensor([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1])


In [162]:
train_load = DataLoader(train_data, batch_size=20, shuffle=False)

In [163]:
len(train_data)

198250

In [118]:
counter = 0
for batch in train_load:
    sent1, sent2, label = batch[0], batch[1], batch[2]
    for x,y,z in zip(sent1,sent2,label):
        counter += 1
        #print(x,y,z)
        
    
print(counter)

14406


In [157]:
sent1[0]

[127253,
 115589,
 127753,
 21581,
 132065,
 114276,
 35971,
 100084,
 107005,
 23222,
 89890,
 133371,
 21581,
 112329,
 134465,
 35971,
 4591,
 55090]

In [188]:
sent1_train_loader = DataLoader(dataset=DebatesSet(sent1, Y_train), batch_size=100, shuffle=False)

In [197]:
sent_1_data_set[127253]

([60337, 4591, 42542, 78053, 28455, 3564, 66991], 0)

In [198]:
b
c

tensor([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 0])

In [180]:
train_loader = DataLoader(dataset=train_data, batch_size=20, shuffle=False)

In [129]:
sent2_train_loader = DataLoader(dataset=DebatesSet(sent2, Y_train), batch_size=1, shuffle=False)

In [130]:
sent1_batch_1 = next(iter(sent1_train_loader))

In [131]:
sent2_batch_2 = next(iter(sent2_train_loader))

In [132]:
sent1_batch_1

[[tensor([127253]),
  tensor([115589]),
  tensor([127753]),
  tensor([21581]),
  tensor([132065]),
  tensor([114276]),
  tensor([35971]),
  tensor([100084]),
  tensor([107005]),
  tensor([23222]),
  tensor([89890]),
  tensor([133371]),
  tensor([21581]),
  tensor([112329]),
  tensor([134465]),
  tensor([35971]),
  tensor([4591]),
  tensor([55090])],
 tensor([0])]

In [133]:
sent2_batch_2

[[tensor([86841]),
  tensor([21581]),
  tensor([83259]),
  tensor([129688]),
  tensor([98731]),
  tensor([136265]),
  tensor([92593]),
  tensor([145901]),
  tensor([74764]),
  tensor([38454]),
  tensor([69022]),
  tensor([146758]),
  tensor([71036]),
  tensor([2487]),
  tensor([21581]),
  tensor([40726])],
 tensor([0])]

147143

In [ ]:
for batch_idx, (sent1, sent2, y) in enumerate(train_loader):
    print(y)
    break

In [ ]:
for x in train_loader:
    print(x)
    break

In [ ]:
import torch
from torch.utils.data import TensorDataset

In [ ]:
nb_samples = 100
features = torch.randn(nb_samples, 10)
labels = torch.empty(nb_samples, dtype=torch.long).random_(10)
adjacency = torch.randn(nb_samples, 5)
laplacian = torch.randn(nb_samples, 7)

dataset = TensorDataset(features, labels, adjacency, laplacian)
loader = DataLoader(
    dataset,
    batch_size=2
)

for batch_idx, (x, y, a, l) in enumerate(loader):
    print(x.shape, y.shape, a.shape, l.shape)

I need to make two instances of the dataloader for each sentence 
in the training I can have two outputs, out1, out2
concatenate the hidden for each output and feed to classifier 
